In [ ]:
import os
import re
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

# códigos CIE-10 a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10, E11, E12, E13, E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00, C01, … C97

# Función para saber si un código AFEC entra en diabetes o cáncer
def es_diabetes_o_cancer(codigo: str) -> bool:
    if not isinstance(codigo, str):
        return False
    codigo = codigo.strip().upper()
    # extraer prefijo de 3 caracteres
    pref = codigo[:3]
    return (pref in prefijos_diabetes) or (pref in prefijos_cancer)

# Procesar cada archivo .csv en la carpeta
for nombre in os.listdir(ruta_entrada):
    if not nombre.lower().endswith(".csv"):
        continue

    # extraer año del nombre (p.ej. “2015” en “…2015.csv”)
    m = re.search(r"(20\d{2})", nombre)
    if not m:
        continue
    año = int(m.group(1))
    if año < 2013 or año > 2023:
        continue

    ruta_archivo = os.path.join(ruta_entrada, nombre)
    print(f"Procesando {nombre}…")

    # leer y filtrar
    df = pd.read_csv(ruta_archivo, dtype=str, encoding="latin1", low_memory=False)
    if "AFEC" not in df.columns:
        print(" Columna AFEC no encontrada, se salta este archivo.")
        continue

    df_filtrado = df[df["AFEC"].apply(es_diabetes_o_cancer)].copy()

    # guardar el CSV único por año
    nombre_salida = f"AFECCIONES{año}.csv"
    ruta_guardado = os.path.join(ruta_salida, nombre_salida)
    df_filtrado.to_csv(ruta_guardado, index=False, encoding="utf-8")
    print(f"  → Guardado: {nombre_salida} ({len(df_filtrado)} registros)")

print(" Todos los archivos 2013–2023 filtrados y guardados.")


In [ ]:
import os
import re
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

# códigos CIE-10  a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10–E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00–C97

def es_diabetes_o_cancer(codigo: str) -> bool:
    if not isinstance(codigo, str):
        return False
    pref = codigo.strip().upper()[:3]
    return (pref in prefijos_diabetes) or (pref in prefijos_cancer)

# Procesar archivos .txt de 2019–2023
for nombre in os.listdir(ruta_entrada):
    if not nombre.lower().endswith(".txt"):
        continue

    m = re.search(r"(20\d{2})", nombre)
    if not m:
        continue
    año = int(m.group(1))
    if año < 2019 or año > 2023:
        continue

    ruta_archivo = os.path.join(ruta_entrada, nombre)
    print(f"Procesando {nombre}…")

    # Leer como CSV con separador coma
    df = pd.read_csv(ruta_archivo, dtype=str, encoding="latin1",
                     low_memory=False, sep=",", on_bad_lines='skip')
    if "AFEC" not in df.columns:
        print("  Columna AFEC no encontrada, se salta.")
        continue

    df_filtrado = df[df["AFEC"].apply(es_diabetes_o_cancer)].copy()

    nombre_salida = f"AFECCIONES{año}.csv"
    ruta_guardado = os.path.join(ruta_salida, nombre_salida)
    df_filtrado.to_csv(ruta_guardado, index=False, encoding="utf-8")
    print(f"  → Guardado: {nombre_salida} ({len(df_filtrado)} registros)")

print(" Archivos .txt 2019–2023 filtrados y guardados.")


In [ ]:
import os
import re
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

# Prefijos CIE-10 a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10–E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00–C97

def es_diabetes_o_cancer(codigo: str) -> bool:
    if not isinstance(codigo, str):
        return False
    pref = codigo.strip().upper()[:3]
    return (pref in prefijos_diabetes) or (pref in prefijos_cancer)

# Procesar .txt de 2020–2023 
for nombre in os.listdir(ruta_entrada):
    if not nombre.lower().endswith(".txt"):
        continue

    m = re.search(r"(20\d{2})", nombre)
    if not m:
        continue
    año = int(m.group(1))
    if año < 2020 or año > 2023:
        continue

    ruta_archivo = os.path.join(ruta_entrada, nombre)
    print(f"Procesando {nombre}…")

    # Leer como CSV con separador '|'
    df = pd.read_csv(
        ruta_archivo,
        sep="|",
        dtype=str,
        encoding="latin1",
        low_memory=False,
        on_bad_lines="skip"
    )

    col = "cod_cie_comorbilidad"
    if col not in df.columns:
        print(f"   Columna {col} no encontrada en {nombre}, se omite.")
        continue

    # Filtrar y guardar
    df_filtrado = df[df[col].apply(es_diabetes_o_cancer)].copy()
    salida = f"AFECCIONES{año}.csv"
    df_filtrado.to_csv(
        os.path.join(ruta_salida, salida),
        index=False,
        encoding="utf-8"
    )
    print(f"  → Guardado {salida} con {len(df_filtrado)} registros.")

print(" Archivos 2020–2023 procesados y guardados.")


In [ ]:
import os
import re
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

# Codigos CIE‑10 a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10–E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00–C97

def es_diabetes_o_cancer(codigo: str) -> bool:
    if not isinstance(codigo, str):
        return False
    pref = codigo.strip().upper()[:3]
    return (pref in prefijos_diabetes) or (pref in prefijos_cancer)

# Procesar .txt de 2021–2023 
for nombre in os.listdir(ruta_entrada):
    if not nombre.lower().endswith(".txt"):
        continue

    # Extraer año del nombre de archivo
    m = re.search(r"(20\d{2})", nombre)
    if not m:
        continue
    año = int(m.group(1))
    # Solo 2021–2023
    if año < 2021 or año > 2023:
        continue

    ruta_archivo = os.path.join(ruta_entrada, nombre)
    print(f"Procesando {nombre}…")

    # Leer usando '|' como separador
    df = pd.read_csv(
        ruta_archivo,
        sep="|",
        dtype=str,
        encoding="latin1",
        low_memory=False,
        on_bad_lines="skip"
    )

    # Validar columna AFEC
    if "AFEC" not in df.columns:
        print(f" Columna AFEC no encontrada en {nombre}, se omite.")
        continue

    # Filtrar registros de diabetes y cáncer
    df_filtrado = df[df["AFEC"].apply(es_diabetes_o_cancer)].copy()

    # Guardar un único CSV por año
    nombre_salida = f"AFECCIONES{año}.csv"
    ruta_guardado = os.path.join(ruta_salida, nombre_salida)
    df_filtrado.to_csv(ruta_guardado, index=False, encoding="utf-8")

    print(f"  → Guardado {nombre_salida} ({len(df_filtrado)} registros)")

print(" Archivos 2021–2023 filtrados y guardados correctamente.")


In [ ]:
import os
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

# codigoss CIE‑10 a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10–E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00–C97

def es_diabetes_o_cancer(codigo: str) -> bool:
    if not isinstance(codigo, str):
        return False
    pref = codigo.strip().upper()[:3]
    return (pref in prefijos_diabetes) or (pref in prefijos_cancer)

# Archivo 2017 (CSV)
archivo_2017 = "AFECCIONES2017.csv"  
ruta_archivo = os.path.join(ruta_entrada, archivo_2017)
print(f"Procesando {archivo_2017}…")

# Leer CSV con separador comma
df = pd.read_csv(
    ruta_archivo,
    dtype=str,
    encoding="latin1",
    low_memory=False,
    on_bad_lines="skip"
)

# Verificar columna AFEC
if "AFEC" not in df.columns:
    raise ValueError(f"La columna AFEC no se encontró en {archivo_2017}")

# Filtrar registros
df_filtrado = df[df["AFEC"].apply(es_diabetes_o_cancer)].copy()

# Guardar CSV limpio
nombre_salida = "AFECCIONES2017_filtrado.csv"
ruta_guardado = os.path.join(ruta_salida, nombre_salida)
df_filtrado.to_csv(ruta_guardado, index=False, encoding="utf-8")

print(f"→ Guardado {nombre_salida} con {len(df_filtrado)} registros")
print(" Archivo 2017 procesado correctamente.")


In [ ]:
import os
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

archivo_2017 = "AFECCIONES2017.csv"  
ruta_archivo = os.path.join(ruta_entrada, archivo_2017)

# Leer sólo la cabecera para ver columnas
df_header = pd.read_csv(ruta_archivo, nrows=0, encoding="latin1")
print("Columnas originales:", list(df_header.columns))

# Normalizar nombres
cols_norm = [c.strip().upper() for c in df_header.columns]
# Buscar columna que contenga 'AFEC'
candidatas = [orig for orig, nor in zip(df_header.columns, cols_norm) if "AFEC" in nor]
if not candidatas:
    raise ValueError(f"No se encontró ninguna columna que contenga 'AFEC' en {archivo_2017}. Columnas disponibles: {cols_norm}")
col_afec = candidatas[0]
print(f"Usaremos la columna: '{col_afec}'")

# Ahora leer todo el df y filtrar
df = pd.read_csv(
    ruta_archivo,
    dtype=str,
    encoding="latin1",
    low_memory=False,
    on_bad_lines="skip"
)

# Prefijos para diabetes y cáncer
pref_diab = [f"E1{d}" for d in range(0, 5)]
pref_canc = [f"C{str(i).zfill(2)}" for i in range(0, 98)]
def es_tipo(c):
    if isinstance(c, str):
        p = c.strip().upper()[:3]
        return p in pref_diab or p in pref_canc
    return False

df_filtrado = df[df[col_afec].apply(es_tipo)].copy()

# Guardar resultado
salida = "AFECCIONES2017_filtrado.csv"
df_filtrado.to_csv(os.path.join(ruta_salida, salida), index=False, encoding="utf-8")
print(f"→ Guardado '{salida}' con {len(df_filtrado)} registros")


In [ ]:
import os
import pandas as pd

# Rutas
ruta_entrada = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Egresos Hospitalarios\Afecciones"
ruta_salida  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
os.makedirs(ruta_salida, exist_ok=True)

# Prefijos CIE‑10 a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10–E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00–C97

def es_diabetes_o_cancer(codigo: str) -> bool:
    if not isinstance(codigo, str):
        return False
    pref = codigo.strip().upper()[:3]
    return (pref in prefijos_diabetes) or (pref in prefijos_cancer)


archivo_2017 = "AFECCIONES2017.csv"
ruta_archivo = os.path.join(ruta_entrada, archivo_2017)

print(f"Procesando {archivo_2017}…")

# Leer usando '|' como separador
df = pd.read_csv(
    ruta_archivo,
    sep="|",
    dtype=str,
    encoding="latin1",
    low_memory=False,
    on_bad_lines="skip"
)

# Verificar columna AFEC
if "AFEC" not in df.columns:
    raise ValueError(f"La columna AFEC no se encontró en {archivo_2017}. Columnas disponibles: {df.columns.tolist()}")

# Filtrar registros de diabetes y cáncer
df_filtrado = df[df["AFEC"].apply(es_diabetes_o_cancer)].copy()

# Guardar CSV limpio
nombre_salida = "AFECCIONES2017.csv"
ruta_guardado = os.path.join(ruta_salida, nombre_salida)
df_filtrado.to_csv(ruta_guardado, index=False, encoding="utf-8")

print(f"→ Guardado {nombre_salida} con {len(df_filtrado)} registros")


In [ ]:
import pandas as pd
import os

# Rutas
input_dir  = r"C:\Users\jesus\OneDrive\Desktop\Archivos de Salud 2010-2023\Datos Limpios\Datos limpios Afecciones"
output_dir = r"C:\Users\jesus\OneDrive\Desktop\DATOS LIMPIOS 2013-2023\AFECCIONES\CANCERDIABETES"
os.makedirs(output_dir, exist_ok=True)

# codigos CIE‑10 a filtrar
prefijos_diabetes = [f"E1{d}" for d in range(0, 5)]              # E10–E14
prefijos_cancer   = [f"C{str(i).zfill(2)}" for i in range(0, 98)] # C00–C97

# Procesar cada año
for year in range(2013, 2024):
    
    in_path = os.path.join(input_dir, f"Afecciones{year}.csv")
    if not os.path.isfile(in_path):
        print(f"  No se encontró: {in_path}")
        continue

    # Cargar datos
    df = pd.read_csv(in_path)

    # Filtrar por diabetes y cáncer
    mask_diabetes = df['AFECCION'].str.startswith(tuple(prefijos_diabetes))
    mask_cancer   = df['AFECCION'].str.startswith(tuple(prefijos_cancer))

    df_diabetes = df[mask_diabetes]
    df_cancer   = df[mask_cancer]

    # Guardar resultados
    out_diab = os.path.join(output_dir, f"AFECCIONESDIABETES{year}.csv")
    out_canc = os.path.join(output_dir, f"AFECCIONESCANCER{year}.csv")

    df_diabetes.to_csv(out_diab, index=False)
    df_cancer.to_csv(out_canc, index=False)

    # Mostrar conteo
    print(f"{year} → Diabetes: {len(df_diabetes)} filas | Cáncer: {len(df_cancer)} filas")
